In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [ ]:
police = pd.read_csv('police_with_neighborhood_stats (1).csv')

In [ ]:
police["Incident Hour"] = police["Incident Time"].apply(lambda x : x[0:2]).astype(int)

def time_category(time):
    if 0 <= time < 3:
        return "Late Night"
    elif 3 <= time < 6:
        return "Middle Night"
    elif 6 <= time < 9:
        return "Morning"
    elif 9 <= time < 12:
        return "Late Morning"
    elif 12 <= time < 15:
        return "Afternoon"
    elif 15 <= time < 18:
        return "Late Afternoon"
    elif 18 <= time < 21:
        return "Evening"
    elif 21 <= time < 24:
        return "Night"
def new_time_category(time):
   if time == 12 or time == 17 or time == 16 or time == 0 or time == 18:
       return "First High Time"
   elif time == 15 or time == 13 or time == 19:
       return "Second High Time"
   elif time == 14 or time == 20 or time == 10 or time == 22 or time == 11 or time == 21:
       return "Third High Time"
   elif time == 23 or time == 9 or time == 8:
       return "Fourth High Time"
   elif time == 1 or time == 2 or time == 7 or time == 3:
       return "Fifth High Time"
   elif time == 4 or time == 6 or time == 5:
       return "Sixth High Time"

police["Time Category"] = police["Incident Hour"].apply(time_category)
police["New Time Category"] = police["Incident Hour"].apply(new_time_category)
police = police.join( pd.get_dummies(police["Time Category"]), how = 'outer')
police = police.join( pd.get_dummies(police["New Time Category"]), how = 'outer')

In [ ]:
police["Incident Day"] = pd.to_datetime(police['Incident Datetime']).dt.day
police["First Week"] = police["Incident Day"].isin([1,2,3,4,5,6,7])
police["Second Week"] = police["Incident Day"].isin([8,9,10,11,12,13,14])

police["Last Week"] = police["Incident Day"].isin([23,24,25,26,27,28,29,30,31])

In [ ]:
police["Incident Month"] = pd.to_datetime(police["Incident Datetime"]).apply(lambda date : date.month)

def season(month):
    if month == 12 or month == 1 or month == 2:
        return "Winter"
    elif month == 3 or month == 4 or month == 5:
        return "Spring"
    elif month == 6 or month == 7 or month == 8:
        return "Summer"
    elif month == 9 or month == 10 or month == 11:
        return "Fall"
police["Season"] = police["Incident Month"].apply(season)
police = police.join(pd.get_dummies(police["Season"]), how = 'outer')
police = police.join(pd.get_dummies(police["Incident Month"].astype(str)), how = 'outer')
police = police.join(pd.get_dummies(police["Incident Year"]), how = 'outer')


In [ ]:
police = police.join(pd.get_dummies(police["Incident Day of Week"]), how = 'outer')
police = police.join(pd.get_dummies(police["Analysis Neighborhood"]), how = 'outer')

In [ ]:
police = police.join(pd.get_dummies(police["Incident Hour"]), how = 'outer')

In [ ]:
police.groupby(["Incident Description"]).count().sort_values("Incident Datetime", ascending = False)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Incident_Category,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point,SF Find Neighborhoods,Current Police Districts,occ,Percent Female,Single Family Homes Percent,Apartment Buildings Percent,Median Rent,Median Home Value,Median Household Income,Percent in Poverty,Unemployment Rate,Percent Asian,Percent Black/AA,Percent White,Percent Native American/Hawaiian,Percent Latinx,High School Degree or less,College Degree,Graduate Degree/PhD,Foreign Born,Median Age,English Speaking (only),Spanish Speaking (only),Asian Language Speaking,Groupings,Incident Hour,Time Category,...,Outer Richmond,Pacific Heights,Portola,Potrero Hill,Presidio,Presidio Heights,Russian Hill,Seacliff,South of Market,Sunset/Parkside,Tenderloin,Treasure Island,Twin Peaks,Visitacion Valley,West of Twin Peaks,Western Addition,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Incident Description,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Theft, From Locked Vehicle, >$950",4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,...,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821,4821
"Malicious Mischief, Vandalism to Property",2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,...,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069
"Vehicle, Stolen, Auto",1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,...,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898
Battery,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,...,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879,1879
"Theft, Other Property, $50-$200",1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,...,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693,1693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Theft, Coin Operated Machine, >$950",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"Theft, Coin Operated Machine, $50-$200",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"Theft

In [ ]:
sum(police.isna().sum())

0

In [ ]:
def groups(x):
  if x in ['Suspicious','Suspicious Occ', 'Malicious Mischief']:
    return 'sus'
  elif x in ['Offences Against The Family And Children', 'Family Offense']:
    return 'family'
  elif x in ['Homicide', 'Assault', 'Suicide']:
    return 'hurting'
  elif x in ['Courtesy Report', 'Case Closure', 'Warrant']:
    return 'case'
  elif x in ['Non-Criminal', 'Civil Sidewalks', 'Disorderly Conduct', 'Vandalism', 'Gambling', 'Juvenile Offenses']:
    return 'Others'
  elif x in ['Fraud', 'Forgery And Counterfeiting', 'Embezzlement']:
    return 'fraud'
  elif x in ['Fire Report', 'Arson']:
    return 'fire'
  elif x in ['Drug Offense', 'Drug Violation', 'Liquor Laws']:
    return 'drug'
  elif x in ['Weapons Offense', 'Weapons Offence', 'Weapons Carrying Etc']:
    return 'weapons'
  elif x in ['Larceny Theft', 'Burglary', 'Stolen Property', 'Motor Vehicle Theft', 'Motor Vehicle Theft?', 'Lost Property', 'Robbery']:
    return 'theft'
  elif x in ['Vehicle Misplaced', 'Recovered Vehicle', 'Vehicle Impounded', 'Traffic Collision', 'Traffic Violation Arrest']:
    return 'vehicle'
  elif x in ['Other Miscellaneous', 'Miscellaneous Investigation', 'Other', 'Other Offenses']:
    return 'misc'
  elif x in ['Sex Offense', 'Prostitution', 'Rape', 'Human Trafficking (A), Commercial Sex Acts', 'Human Trafficking, Commercial Sex Acts']:
    return 'sex'



g = []
for i in police['Incident_Category']:
  g.append(groups(i))
police['Groupings'] = g

In [ ]:
def broad_groups(x):
  if x in ['Offences Against The Family And Children', 'Family Offense', 
           'Non-Criminal', 'Civil Sidewalks', 'Disorderly Conduct', 'Vandalism', 'Gambling', 'Juvenile Offenses',
           'Malicious Mischief',  'Vehicle Misplaced', 'Recovered Vehicle']:
    return 'low grade offense'
  elif x in ['Fraud', 'Forgery And Counterfeiting', 'Embezzlement', 'Drug Offense', 'Drug Violation', 'Liquor Laws',
             'Suspicious','Suspicious Occ', 'Warrant', 
             'Vehicle Impounded', 'Traffic Collision', 'Traffic Violation Arrest',
             'Larceny Theft', 'Burglary', 'Stolen Property', 'Motor Vehicle Theft', 
             'Motor Vehicle Theft?', 'Lost Property', 'Robbery', 'Fire Report', 'Arson']:
    return 'medium grade offense'
  elif x in ['Homicide', 'Assault', 'Suicide', 'Weapons Offense', 'Weapons Offence', 'Weapons Carrying Etc',
             'Sex Offense', 'Prostitution', 'Rape', 'Human Trafficking (A), Commercial Sex Acts', 
             'Human Trafficking, Commercial Sex Acts']:
    return 'violent'
  # elif x in ['Sex Offense', 'Prostitution', 'Rape', 'Human Trafficking (A), Commercial Sex Acts', 
  #            'Human Trafficking, Commercial Sex Acts']:
  #   return 'sex'
  elif x in ['Other Miscellaneous', 'Miscellaneous Investigation', 'Other', 'Other Offenses', 'Courtesy Report',
             'Case Closure']:
    return 'misc'




bg = []
for i in police['Incident_Category']:
  bg.append(broad_groups(i))
police['Broad Groupings'] = bg

In [ ]:
def new_groups(x):
 if x in ['Theft, From Locked Vehicle, >$950', 'Theft, Other Property, $50-$200',
            'Embezzlement from Dependent or Elder Adult by Caretaker',
            'Theft, Shoplifting, $200-$950', 'Theft, From Building, Att.',
            'Theft, From Person, >$950 (other than Pickpocket)', 'Theft, From Person, $200-$950 (other than Pickpocket)', 'Theft, From Locked Vehicle, $200-$950', 'Burglary, Vehicle (Arrest made)', 'Theft, Pickpocket, $50-$200', 'Theft, From Unlocked Vehicle, >$950', 'Theft, From Locked Vehicle, <$50', 'Theft, Shoplifting, $50-$200', 'License Plate, Stolen', 'Theft, Other Property, >$950', 'Theft, From Building, $200-$950', 'Theft, From Locked Vehicle, Att.', 'Theft, From Locked Vehicle, $50-$200', 'Theft, Shoplifting, <$50', 'Theft, Lost Property, Petty', 'Theft, From Building, $50-$200', 'Theft, From Unlocked Vehicle, $200-$950', 'Theft, Bicycle, Att.', 'Theft, Pickpocket, <$50', 'Theft of Checks or Credit Cards', 'Theft, From Building, >$950', 'Theft, From Person, <$50 (other than Pickpocket)', 'Theft, From Building, <$50', 'Theft, Shoplifting, >$950', 'Burglary, Vehicle, Att. (Arrest made)', 'Theft, Pickpocket, $200-$950', 'Theft, Other Property, $200-$950', 'Theft, From Unlocked Vehicle, $50-$200', 'Theft, Bicycle, $200-$950', 'Theft of Computers or Cell Phones', 'Theft, Petty, with Prior Conviction', 'Theft, Bicycle, >$950', 'Theft, Bicycle, $50-$200', 'Theft from Merchant or Library', 'Theft, From Person, $50-$200 (other than Pickpocket)', 'Theft, Shoplifting, Att.', 'Theft, Other Property, <$50', 'Theft, Lost Property, Grand', 'Theft, Pickpocket, >$950', 'Theft, Vehicle Strip, >$950', 'Theft of Other Property, Att.', 'Theft, Vehicle Strip, $200-$950', 'Theft of Animals (general)', 'Theft, From Unlocked Vehicle, <$50', 'Theft, From Unlocked Vehicle, Att.', 'License Plate or Tab, Theft of', 'Theft, Coin Operated Machine, >$950', 'Theft, Drunk Roll, >$950',
            'Theft, Vehicle Strip, <$50', 'Theft, Drunk Roll, $200-$950', 'Theft, Coin Operated Machine, $200-$950', 'Theft of Utility Services', 'Theft, Purse Snatch, <$50', 'Theft, Motorcycle Strip, $200-$950', 'Theft, Vehicle Strip, Attempted', 'Theft, Vehicle Strip, $50-$200', 'Theft, Purse Snatch, $200-$950', 'Theft, From Person, Att. (other than Pickpocket)', 'Looting during State of Emergency', 'Theft, Bicycle, <$50, no serial number', 'Theft, Purse Snatch, >$950', 'Theft of Telecommunication Services, incl. Clone Phone', 'Theft, Drunk Roll, $50-$200', 'Theft, Coin Operated Machine, $50-$200', 'Theft, Purse Snatch, $50-$200', 'Theft, Boat', 'Theft, Motorcycle Strip, >$950', 'Burglary, Residence, Forcible Entry', 'Burglary, Apartment House, Forcible Entry', 'Burglary, Hot Prowl, Forcible Entry', 'Burglary, Other Bldg., Forcible Entry', 'Burglary, Apartment House, Unlawful Entry', 'Burglary, Other Bldg., Unlawful Entry', 'Burglary, Apartment House, Att. Forcible Entry', 'Burglary, Residence, Unlawful Entry', 'Burglary, Store, Unlawful Entry', 'Burglary, Non-residential, Forcible Entry', 'Burglary, Hot Prowl, Unlawful Entry', 'Burglary, Non-residential, Unlawful Entry', 'Burglary, Warehouse, Forcible Entry', 'Burglary, Safe, Store', 'Burglary, Store, Forcible Entry', 'Burglary, Residence, Att. Forcible Entry', 'Burglary, Residence Under Constr, Forcible Entry', 'Burglary, Other Bldg. Under Constr., Forcible Entry', 'Burglary, Warehouse, Unlawful Entry', 'Burglary, Other Bldg. Under Constr., Unlawful Entry', 'Burglary, Hotel Room, Unlawful Entry', 'Burglary, Flat, Unlawful Entry', 'Burglary, Store, Att. Forcible Entry', 'Burglary, Other Bldg., Att. Forcible Entry',
            'Burglary, Non-residential, Att. Forcible', 'Burglary, Residence Under Constr, Unlawful Entry', 'Burglary, Hot Prowl, Att. Forcible Entry', 'Burglary, Flat Under Constr., Att. Forcible', 'Burglary, Flat Under Constr., Forcible Entry', 'Burglary, Apt Under Constr., Unlawful Entry', 'Burglary, Apt Under Constr., Forcible Entry',
            'Burglary, Other Bldg. Under Constr., Att. Forcible', 'Burglary, Warehouse, Att. Forcible Entry', 'Burglary, Hotel Room, Forcible Entry', 'Burglary, Flat, Forcible Entry', 'Burglary, Hotel Under Constr., Forcible Entry', 'Burglary, Safe, Residence', 'Burglary, Safe, Warehouse', 'Burglary, Apt Under Constr., Att. Forcible', 'Burglary, Warehouse Under Constr, Unlawful Entry', 'Burglary, Flat Under Constr., Unlawful Entry', 'Burglary, Residence Under Constr, Att. Forcible', 'Burglary, Flat, Att. Forcible Entry', 'Burglary, Safe, Other', 'Burglary, Hotel Under Constr., Unlawful Entry', 'Stolen Property, Possession with Knowledge, Receiving', 'Stolen Computer, Possession', 'Money, Property or Labor, Fraudulently Obtaining', 'Stolen Cellular Phone, non-cloned, Possession', 'Stolen Access Card, Possession', 'Stolen Electronics, Possession', 'Stolen Checks, Possession', 'Lost Property', 'Embezzlement (general)', 'Embezzlement, Grand Theft By Employee', 'Embezzlement, Grand Theft', 'Embezzlement, Petty Theft By Employee', 'Embezzlement, Petty Theft', 'Embezzlement, Petty Theft, Leased Property', 'Vehicle, Embezzled', 'Vehicle, Rental, Failure to Return', 'Embezzlement, Grand Theft Leased Property', 'Embezzlement, Grand Theft By Brokers/Agents', 'Counterfeit Money or Coins', 'Checks, False, Making, Uttering or Possessing', 'Forgery, Checks, Felony', 'Access Card or Signature, Forgery of', 'Checks, Blank or Forged, Fraudulent Possession', 'Forgery & Counterfeiting (general)', 'Counterfeit/Forgery, Poss Drivers Lic Or Id Card', 'Counterfeit Coins Or Notes, Possessing or Receiving', 'Vehicle Registration, False Evidence of', 'Counterfeiting, Coins Or Notes',
            'Checks or Legal Instruments, Uttering Forged', 'Identification Card or Drivers License, Falsify or Display', 'VIN, Alter Or Remove', 'Access Card, incl. Credit, Phone, ATM, Fraudulent Use of', 'False Personation', 'False Personation to Receive Money or Property', 'Fraudulent Game or Trick, Obtaining Money or Property', 'Fraudulent Use Of Automated Teller Card', 'Access Card Information, Theft of', 'Checks, Making or Uttering with Insufficient Funds', 'Checks, Money Orders, Completed, Fraudulent Possession', 'Credit Application, Fraudulent', 'False Personation and Cheat Crimes (general)', 'Theft, Trick And Device, Felony', 'Computer System, Accessing, Copying, or Damaging', 'Theft, False Pretenses, Felony', 'Access Card Counterfeiting Machinery, Possession of', 'Checks, Theft/Non-sufficient Funds, Felony', 'Identification, Government, Possess, Make or Sell False', 'Access Card Information, Publication of', 'Theft, Trick & Device, Misd.', 'Cellular or Cordless Phone Communications, Intercepting', 'Telephone or Telegraph Message, Sending False', 'Food Stamps, Misuse of', 'Access Card, Possession of Lost or Stolen, with Intent to Use, Transfer, or Sell', 'False Claims, Presenting to Government', 'Privacy, Invasion of (general)', 'Theft, False Pretenses, Misd.', 'Falsification of Medical Records', 'Audiovisual (Video or Sound) Recordings, Unauthorized', 'Theft, Trick And Device, Att.', 'Eavesdropping Devices, Sale or Use',
            'Financial Statements, False', 'Insured Property, Destruction to Defraud Insurer', 'Park Code Violation (general)', 'Trash, Placing On The Street', 'Alcohol, Consuming In Public View', 'Municipal Code Violation (general)', 'Municipal Police Code Violation (general)', 'Defrauding Gas Station/Taxi Driver', 'Throwing Object at Common Carrier, Passenger or Freight', 'Obstructions On Streets, Sidewalks', 'Loitering and Peeking into Inhabited Structure', 'License Plate, Lost', 'License Plate, Found', 'Peddling Without A License', 'Fire Code Violation (general)',
            'Permit Violation, Police (general)', 'Permit Violation, Entertainment', 'Federal Law Violation (general)', 'Hotel Violations', 'Public Transit, Acts Against', 'Burglary Tools, Possession Of', 'Conspiracy', 'Investigative Detention', 'Resisting, Delaying, or Obstructing Peace Officer Duties',
            'Dissuading Witness, Victim', 'Identification To Peace Officer, False',
            'Driving, License Suspended or Revoked', 'Trespassing',
            'Driving While Under The Influence Of Drugs', 'Probation Violation',
            'Injuring Phone, Cable TV, or Other Line', 'Driving, No License Issued',
            'Lodging Without Permission', 'Unusual Occurrence', 'Driving, Reckless',
            'Evading a Police Officer Recklessly', 'Driving, Speeding',
            'Defrauding Providers of Food, Fuel, Services, or Accommodations',
            'Threat or Force to Resist Executive Officer', 'Burglary Tools, Possession Of w/prior',
            'Evading a Police Vehicle or Bicycle', 'Parole Violation, Adult',
            'Narcotics Paraphernalia, Possession of', 'Methamphetamine Offense', 'Cocaine, Possession For Sale', 'Marijuana, Transporting', 'Cocaine, Base/rock, Possession For Sale', 'Methamphetamine, Possession For Sale', 'Heroin, Possession For Sale', 'Cocaine, Sale', 'Marijuana, Possession For Sale', 'Cocaine, Base/rock, Sale', 'Marijuana, Cultivating/Planting', 'Methamphetamine, Transportation', 'Cocaine Offense', 'Controlled Substance Offense', 'Heroin Offense', 'Controlled Substance, Possession For Sale', 'Controlled Substance, Sale', 'Loitering Where Narcotics are Sold/Used', 'Cocaine, Base/rock Offense', 'Hallucinogenic, Possession For Sale', 'Marijuana, Sales', 'Controlled Substance, Under the Influence of', 'Heroin, Sales', 'Controlled Substance, Transportation', 'Marijuana Offense', 'Opiates, Sale', 'Methamphetamine, Sale', 'Hallucinogenics Offense', 'Money Offense Related to Narcotics Trafficking', 'Opium Derivative, Possession For Sale', 'Opiates, Possession For Sale', 'Opiates Offense', 'Controlled Substance Violation, Loitering for', 'Methadone Offense', 'Methadone, Possession For Sale', 'Hallucinogenic, Sale', 'Prescription, Forge Or Alter (11368 H&S)', 'Amphetamine, Transportation', 'Amphetamine, Possession For Sale', 'Liquor Law Violation (general)', 'Alcohol, Unlawful Transportation Of', 'Alcohol, Possession Of By Minor', 'Alcohol, Sale Of To Minor', 'Hallucinogenic,  Possession For Sale', 'Drugs, Under Influence  in a Public Place',
            'Dissuading/Threatening Of A Witness', 'Public Nuisance, Maintaining After Notification', 'Driving While Under The Influence Of Alcohol', 'Loitering Without Lawful Business With Owner or Occupant', 'State Code Felony (general)', 'Resisting Peace Officer, causing Their Serious Injury or Death', 'False Imprisonment', 'Fireworks, Possession of any within SF, incl. "Safe & Sane"', 'Fireworks, Exploding', 'Maintaining A Public Nuisance', 'State Code Misdemeanor (general)',
            'Firefighter, Interfering with', 'Police Broadcast, Interception to Commit Crime', 'Public Transit Crimes - Infractions', 'Extortion or Blackmail', 'Impersonating A Police Officer', 'Rail Transit, Entering, Remaining, or Hindering Operation of', 'False Report of Emergency', 'Vehicle, Open Container Of Alcohol In', 'Extortion, Attempted', 'Interfering, W/ Police Officer', 'Prisoner, Rescuing From Lawful Custody', 'Vehicle, Throwing Substance At', 'Impersonating Public Utility Member', 'False Report of Felony or Misdemeanor', 'Chop Shop, Own or Operate',
            'Threatening School or Public Employee', 'Abortion, Unlawful', 'Driving, Failure To Heed Red Light And Siren', 'Weapon, Taking or Attempting to Take from Peace Officer', 'Legal Instruments, Uttering False Checks, etc. (misd.)', 'Contraband, Taking Into A Reformatory', 'Lotteries (general)', 'Vehicle, Stolen, Auto', 'Vehicle, Stolen, Motorcycle', 'Vehicle, Stolen, Truck', 'Vehicle, Stolen, Other Vehicle', 'Vehicle, Stolen, Attempted', 'Vehicle, Stolen, Bus', 'Vehicle, Stolen, Mobile Home or House Trailer', 'Auto, Grand Theft of', 'Traffic Collision, Hit & Run, Injury', 'Traffic Collision', 'Traffic Collision, Hit & Run, Property Damage', 'Traffic Violation Arrest', 'Vehicle, Tampering', 'Vehicle, Impounded', 'Vehicle, Misplaced', 'Restraining Order Notification/Service of Restraining Order', 'Stay Away Order Violation, DVU related', 'Violation of Restraining Order', 'Violation of Emergency Protective Order', 'Violation of Stay Away Order', 'Stalking', 'Tobacco Products, Selling or Furnishing to Minor', 'ATM-related Crime (secondary code only)', 'Dog, Barking', 'Resisting, Delaying, or Obstructing  Peace Officer Duties', 'Burglary, Safe,  Store', 'Fireworks, Possession of any within SF,  incl. "Safe & Sane"', 'Burglary, Safe,  Warehouse', 'Burglary, Safe,  Residence', 'Privacy, Invasion of  (general)']:
   return 'Crime Without Endangerment'
 elif x in ['Miscellaneous Investigation', 'Assault, Aggravated, W/ Knife', 'Assault, Aggravated, W/ Other Weapon', 'Battery', 'Assault, Aggravated, W/ Gun', 'Assault, Aggravated, W/ Force', 'Battery, former Spouse or Dating Relationship', 'False Imprisonment', 'Assault, Mayhem, W/ Force', 'Battery with Serious Injuries', 'Assault, Non-Aggravated', 'Battery, Sexual', 'Shooting into Inhabited Dwelling or Occupied Vehicle', 'Battery, Of A Police Officer', 'Assault, Att. Homicide, W/ Other Weapon', 'Assault, W/ Caustic Chemical to Injure or Disfigure', 'Assault, Aggravated, On Police Officer, W/ Force', 'Firearm, Discharging in Grossly Negligent Manner', 'Assault, Att. Homicide, W/ Knife', 'Assault, Att. Homicide, W/ Force', 'Assault, Non-Aggravated, Att.', 'Assault, Att. Homicide, W/ Gun', 'Firearm, Discharging at Unoccupied Bldg, Vehicle or Aircraft', 'Assault, Att. Mayhem, W/ Gun', 'Assault, Att. Mayhem, W/ Force', 'Assault, Aggravated, On Police Off., W/ Other Weapon', 'Firearm, Discharging at Occupied Bldg, Vehicle, or Aircraft', 'Firearm, Discharging At An Inhabited Dwelling', 'Assault, Aggravated, On Police Officer, W/ Knife', 'Assault, Mayhem, W/ Knife', 'Firearm, Negligent Discharge', 'Robbery, W/ Force', 'Robbery, W/ Knife', 'Robbery, Att., W/ Force', 'Robbery, Street or Public Place, W/ Force', 'Robbery, Att., Street or Public Place, W/ Knife', 'Shoplifting, Force against Agent', 'Robbery, Att., Chain Store, W/ Force', 'Robbery, Chain Store, W/ Knife', 'Robbery, W/ Other Weapon', 'Robbery, Att., Street or Public Place, W/ Force', 'Robbery, Street or Public Place, W/ Gun', 'Robbery, Chain Store, W/ Force', 'Robbery, Comml. Establ. W/ A Knife', 'Robbery, W/ Gun', 'Carjacking with a Gun', 'Robbery, Att., W/ Gun', 'Robbery, ATM, Gun, Att.', 'Robbery, Residence, W/ Gun', 'Robbery, Comml. Establ., W/ Force', 'Robbery, Att., Comm. Establ., W/ Force', 'Robbery, Att., Comm. Establ. With Other Weapon', 'Robbery, Chain Store, W/ Gun', 'Robbery, Service Station, W/ Gun', 'Robbery, Street or Public Place, W/ Knife', 'Robbery, Street or Public Place, W/ Other Weapon', 'Robbery, Att., W/ Other Weapon', 'Robbery, Att., Chain Store, W/ Other Weapon', 'Robbery, Att., Street or Public Place, W/ Gun', 'Robbery, Residence, W/ Knife', 'Carjacking with Bodily Force', 'Robbery, Comml. Establ. W/ Weapon', 'Robbery, Chain Store, W/ Other Weapon', 'Robbery, Att., Residence, W/ Knife', 'Robbery, ATM, Gun', 'Robbery, Att., Bank, W/ Force', 'Robbery, Vehicle for Hire, Att., W/ Other Weapon', 'Robbery, Residence, W/ Force', 'Robbery, ATM, Force, Att.', 'Carjacking with a Knife', 'Robbery, ATM, Knife', 'Carjacking with a Dangerous Weapon', 'Robbery, Att., Street or Public Place, W/ Other Weapon', 'Robbery, ATM, Force.', 'Robbery, Att., W/ Knife', 'Robbery, Att., Bank, W/ Other Weapon', 'Robbery, Comml. Establ., W/ Gun', 'Robbery, ATM, Other Weapon', 'Robbery, Service Station W/ Other Weapon', 'Robbery, Residence, W/ Other Weapon', 'Robbery, Bank, W/ Force', 'Robbery, Bank, W/ Other Weapon', 'Robbery, Att., Residence, W/ Gun', 'Robbery, ATM, Knife, Att.', 'Firearm, Armed While Possessing Controlled Substance', 'Firearm, Loaded, in Vehicle, Possession or Use', 'Firearm, Possession of Loaded', 'Firearm, Possession By Prohibited Person', 'Firearm, Tampering With Marks', 'Weapon, Prohibited, Possession, Sale, Mfg., Import', 'Weapon, Assault, Possession, Manufacture, or Sale', 'Ammunition, Poss. By Prohibited Person', 'Weapon, Assault, Registration or Transfer Violation', 'Switchblade Knife, Possession', 'Weapon, Deadly, Carrying with Intent to Commit Assault', 'Firearm, Carrying Loaded with Intent to Commit Felony', 'Weapon, Possess or Bring Other on School Grounds', 'Weapon, Tear Gas, Possession by Prohibited Persons', 'Fire Bomb, Possession or Use', 'Firearm, Possession of While Wearing Mask', 'Weapon, Concealed, Loitering While Carrying', 'Weapon, Carrying Concealed', 'Weapon, Deadly, Possession W/ Intent To Assault', 'Weapon, Deadly, Imitation or Laser Scope, Exhibiting', 'Firearm, Discharging Within City Limits', 'Caustic Chemical, Possession', 'Weapon, Carrying Concealed with Felony or Narcotics Conviction', 'Explosive Device, Located', 'Destructive Device, Possession', 'Destructive Device, Exploding', 'Malicious Mischief, Vandalism to Property', 'Malicious Mischief, Vandalism to Vehicle', 'Malicious Mischief, Breaking Windows', 'Malicious Mischief, Adult Suspect', 'Malicious Mischief, Graffiti, Real or Personal Property', 'Vandalism or Graffiti on or within 100 ft of Highway', 'Phone Calls, Harassing', 'Vandalism with Noxious Chemical', 'Animal, Cruelty to', 'Vandalism or Graffiti Tools, Possession', 'Malicious Mischief, Tire Slashing', 'Damage To Mail Box', 'Malicious Mischief, Street Cars/Buses', 'Fire Alarm, False', 'Parking Meters, Damage to', 'Malicious Mischief, Breaking Windows With BB Gun', 'Minor Beyond Parental Control', 'Fire Protection Equipment, Tampering with', 'Malicious Mischief, Building Under Constr.', 'Malicious Mischief, Annoying/Fictitious Phone Calls', 'Curfew Violation', 'Graffiti on Government Vehicles or Public Transportation', 'Terrorist Threats', 'Gang Related (secondary only)', 'Disturbing The Peace, Commotion', 'Phone Calls, Threatening', 'Disturbing The Peace', 'Disturbing The Peace, Fighting', 'Incident On School Grounds', 'Drugs, Under Influence in a Public Place', 'Alcohol, Under Influence Of In Public Place', 'School Property, Disturbance on', 'Public Nuisance, Committing', 'Phone Calls, Threatening or Annoying', 'Lodging in Park', 'School or Playground, Loitering About', 'Firearm Possession in School Zone', 'Disturbing The Peace, Swearing', 'Loitering, Obstructing', 'Assault on School Employee', 'Trespass within 30 days of Credible Threat', 'School Campus, Remaining Without Consent', 'Unlawful Assembly', 'Disturbing Religious Meetings', 'Engaging In Lewd Act', 'Riot', 'School, Public, Trespass', 'Lynching by Riot', 'Refusing To Disperse Upon Lawful Command', 'Suspicious Occurrence', 'Child, Suspicious Act Towards', 'Suspicious Occurrence, possible Shots Fired', 'Suspicious Act Towards Female', 'Suspicious Vehicle', 'Suspicious Person', 'Shooting, Accidental', 'Air Gun, Possession', 'Theft, Grand, of Firearm', 'Spouse, Cohabitee, Parent of Child in Common, Inflict Injury', 'Elder Adult or Dependent Abuse (not Embezzlement or Theft)', 'Domestic Violence (secondary only)', 'Hate Crime (secondary only)', 'Tarasoff Report', 'EPO or Restraining Order Violation, DVU related', 'Probation Violation, DVU related',  'Terrorizing by Arson or Explosive Device', 'Firearm, Turned In by Public', 'Dog, Bite or Attack', 'Dog, Stray or Vicious', 'Hazardous Materials, Dump any Substance into Water', 'Hazardous Materials, Dumping in Unauthorized Location', 'Suspicious Package', 'Missing Adult', 'Discharging Offensive or Injurious Substance in Public Area', 'Throwing Injurious Substance on Highway', 'Firearm with Altered Identification', 'Fireworks, Throw at Person or Discharge in Crowd', 'Driving While Under The Influence Of Alcohol, w/Injury', 'Bomb Threat or False Report of Bomb', 'Evading a Police Officer, Injury or Death', 'Driving While Under The Influence Of Drugs, w/Injury', 'Kidnapping (Adult victim)', 'Flammable or Explosive Device, Possession',  'Kidnapping during Robbery', 'Kidnapping during Carjacking', 'Kidnapping, Attempted (Adult victim)', 'Robbery,  ATM, Gun, Att.', 'Robbery,  ATM, Gun', 'Robbery,  ATM, Knife, Att.', 'Hazardous Materials,  Dump any Substance into Water', 'Robbery,  ATM, Other Weapon', 'Robbery,  ATM, Force.', 'Robbery,  ATM, Force, Att.', 'Robbery,  ATM, Knife',
            'Human Trafficking (A)-Commerical Sex Acts', 'Pimping', 'Rape, Assault To, W/ Force', 'Rape, Forcible, W/ Force', 'Rape, Spousal', 'Oral Copulation, Unlawful (Adult Victim)', 'Rape, Assault To, W/ Gun', 'Rape, Forcible, W/ Sharp Object', 'Penetration, Forced, with Object', 'Unlawful Sexual Intercourse with Minor', 'Rape, Att., W/ Other Weapon', 'Rape, Att., W/ Force', 'Solicits For Act Of Prostitution', 'Pandering', 'Loitering for Purpose of Prostitution', 'Soliciting to Visit House Of Prostitution', 'Solicits Lewd Act', 'House of Prostitution, Keeping or Residing in', 'Soliciting or Engaging In Lewd Conduct']:
   return 'Assault/Endangerment'
 elif x in ['Aided Case', 'Aided case, Naloxone Deployment', 'Aided Case -Property for Destruction', 'Aided Case, Injured or Sick Person', 'Aided Case, Sick Person', 'Death Report, Cause Unknown', 'Death Report, Natural Causes', 'Death, Non-manslaughter Vehicle Accident', 'Death, Accidental', 'Homicide, W/ Gun', 'Homicide, W/ Other Weapon', 'Manslaughter, Vehicular', 'Homicide', 'Suicide By Ingestion, Att.', 'Suicide By Strangulation/Hanging', 'Suicide', 'Suicide By Laceration/Stabbing, Att.', 'Suicide By Asphyxiation, Att.', 'Suicide By Laceration/Stabbing', 'Suicide By Jumping, Att.', 'Suicide, Attempted', 'Suicide By Firearms', 'Suicide By Strangulation/Hanging, Att.', 'Suicide By Firearms, Att.']:
   return 'EMT'
 elif x in ['Arson of Vehicle', 'Arson', 'Arson, Attempted', 'Arson, Inhabited Dwelling', 'Fire, Unlawfully Causing', 'Arson, Portable Toilet', 'Arson, Commercial Building', 'Arson, Vacant Building', 'Arson, Police Vehicle', 'Fire Report']:
   return 'Fire Department'
 elif x in ['Child, Inflicting Physical Pain, Mental Suffering, or Death', 'Obscene Matter, Distribution to Minors', 'Child Abuse, Pornography', 'Sex Offender, Failure To Register for Sexual Assault of Adult', 'Indecent Exposure (Adult victim)', 'Children, Annoy Or Molest', 'Child Abuse, Exploitation', 'Bathroom Hole, Looking through', 'Child Abuse, Sexual', 'Soliciting Minor to Commit Felony', 'Child, Inflicting Injury Resulting in Traumatic Condition', 'Child, Failure To Provide',  'Child, Willful Desertion of', 'Child Concealment, Depriving Lawful Custodian', 'Minor, Contributing To Delinquency of', 'Children, Abandonment & Neglect of (general)', 'Child, Willful Cruelty To', 'Civil Sidewalks, Warning', 'Civil Sidewalks, Citation', 'Civil Sidewalks, Booking', 'Civil Sidewalks, Violation', 'Mental Health Detention']:
   return 'Social Workers'
 elif x in ['Warrant Arrest, Local SF Warrant', 'Warrant Arrest, Enroute To Probation', 'Warrant Arrest, Enroute To Outside Jurisdiction', 'Warrant Arrest, Enroute To Dept. Of Corrections', 'Warrant Arrest, Enroute To Adult Authority', 'Probation Search', 'Post Release Community Supervision', 'Warrant Arrest, Enroute To Parole', 'Warrant Arrest, Enroute To US Marshall', 'Parole Search', 'Search Warrant Service', 'Stay Away or Court Order, Non-DV Related',
            'Case Closure', 'Courtesy Report', 'Found Property', 'Found  Property','Property for Identification', 'Located Property', 'Found Person', 'License Plate, Recovered', 'Vehicle, Recovered, Stolen outside SF', 'Vehicle, Stolen & Recovered', 'Vehicle, Recovered, Motorcycle', 'Vehicle, Recovered, Auto', 'Vehicle, Recovered, Truck', 'Vehicle, Recovered, Other Vehicle', 'Vehicle, Recovered, Mobile Home or House Trailer', 'Vehicle, Recovered, Bus']:
   return 'Administrative'
 
 
gr = []
for i in police['Incident Description']:
   gr.append(new_groups(i))
police['New Groupings'] = gr


In [ ]:
print(list(police.columns))

['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year', 'Incident Day of Week', 'Report Datetime', 'Incident_Category', 'Incident Description', 'Police District', 'Analysis Neighborhood', 'Supervisor District', 'Latitude', 'Longitude', 'point', 'SF Find Neighborhoods', 'Current Police Districts', 'occ', 'Percent Female', 'Single Family Homes Percent', 'Apartment Buildings Percent', 'Median Rent', 'Median Home Value', 'Median Household Income', 'Percent in Poverty', 'Unemployment Rate', 'Percent Asian', 'Percent Black/AA', 'Percent White', 'Percent Native American/Hawaiian', 'Percent Latinx', 'High School Degree or less', 'College Degree', 'Graduate Degree/PhD', 'Foreign Born', 'Median Age', 'English Speaking (only)', 'Spanish Speaking (only)', 'Asian Language Speaking', 'Groupings', 'Incident Hour', 'Time Category', 'New Time Category', 'Afternoon', 'Evening', 'Late Afternoon', 'Late Morning', 'Late Night', 'Middle Night', 'Morning', 'Night', 'Fifth High Time', 'First 

In [ ]:
police.groupby("New Groupings").count()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Incident_Category,Incident Description,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point,SF Find Neighborhoods,Current Police Districts,occ,Percent Female,Single Family Homes Percent,Apartment Buildings Percent,Median Rent,Median Home Value,Median Household Income,Percent in Poverty,Unemployment Rate,Percent Asian,Percent Black/AA,Percent White,Percent Native American/Hawaiian,Percent Latinx,High School Degree or less,College Degree,Graduate Degree/PhD,Foreign Born,Median Age,English Speaking (only),Spanish Speaking (only),Asian Language Speaking,Groupings,Incident Hour,...,Pacific Heights,Portola,Potrero Hill,Presidio,Presidio Heights,Russian Hill,Seacliff,South of Market,Sunset/Parkside,Tenderloin,Treasure Island,Twin Peaks,Visitacion Valley,West of Twin Peaks,Western Addition,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Broad Groupings
New Groupings,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Administrative,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,...,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140
Assault/Endangerment,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,...,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412,14412
Crime Without Endangerment,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,...,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108,34108
EMT,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,...,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496,1496
Fire Department,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,...,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321,321
Social Workers,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,...,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977


In [ ]:
#police = police[police["Groupings"] != 'vehicle']
#police = police[police["Broad Groupings"] != 'misc']
#police = police[police["Incident Description"] != 'Miscellaneous Investigation']
#police = police[police["Incident Description"] != 'Theft, From Locked Vehicle, >$950']

In [ ]:
police.shape[0]

58454

In [ ]:
cols = [2018, 2019, 2020,
        '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9',
        'First Week', 'Second Week', 'Last Week',
        'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday','Wednesday',
        'Bayview Hunters Point', 'Bernal Heights',
       'Castro/Upper Market', 'Chinatown', 'Excelsior',
       'Financial District/South Beach', 'Glen Park', 'Golden Gate Park',
       'Haight Ashbury', 'Hayes Valley', 'Inner Richmond', 'Inner Sunset',
       'Japantown', 'Lakeshore', 'Lincoln Park', 'Lone Mountain/USF', 'Marina',
       'McLaren Park', 'Mission', 'Mission Bay', 'Nob Hill', 'Noe Valley',
       'North Beach', 'Oceanview/Merced/Ingleside', 'Outer Mission',
       'Outer Richmond', 'Pacific Heights', 'Portola', 'Potrero Hill',
       'Presidio', 'Presidio Heights', 'Russian Hill', 'Seacliff',
       'South of Market', 'Sunset/Parkside', 'Tenderloin', 'Treasure Island',
       'Twin Peaks', 'Visitacion Valley', 'West of Twin Peaks',
       'Western Addition', 
        'Percent Female', 'Single Family Homes Percent', 'Apartment Buildings Percent',
        'Median Rent', 'Median Home Value', 'Median Household Income', 'Percent in Poverty', 'Unemployment Rate',
        'Percent Asian', 'Percent Black/AA', 'Percent White', 'Percent Native American/Hawaiian', 'Percent Latinx',
        'High School Degree or less', 'College Degree', 'Graduate Degree/PhD', 'Foreign Born', 'Median Age', 
        'English Speaking (only)', 'Spanish Speaking (only)', 'Asian Language Speaking',
         0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

X = police[cols]
#Normalize features
vals = X.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(vals)
X = pd.DataFrame(x_scaled, columns = X.columns)

2018, 2019, 2020,
        '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9',
        'First Week', 'Second Week', 'Last Week',
        'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday','Wednesday',
        'Bayview Hunters Point', 'Bernal Heights',
       'Castro/Upper Market', 'Chinatown', 'Excelsior',
       'Financial District/South Beach', 'Glen Park', 'Golden Gate Park',
       'Haight Ashbury', 'Hayes Valley', 'Inner Richmond', 'Inner Sunset',
       'Japantown', 'Lakeshore', 'Lincoln Park', 'Lone Mountain/USF', 'Marina',
       'McLaren Park', 'Mission', 'Mission Bay', 'Nob Hill', 'Noe Valley',
       'North Beach', 'Oceanview/Merced/Ingleside', 'Outer Mission',
       'Outer Richmond', 'Pacific Heights', 'Portola', 'Potrero Hill',
       'Presidio', 'Presidio Heights', 'Russian Hill', 'Seacliff',
       'South of Market', 'Sunset/Parkside', 'Tenderloin', 'Treasure Island',
       'Twin Peaks', 'Visitacion Valley', 'West of Twin Peaks',
       'Western Addition', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23

In [ ]:

sum(police["New Groupings"].isna())

0



1.   criterion = 'gini', n_estimators=1000, max_depth=100, min_samples_split=10, min_samples_leaf=1, max_features='auto', oob_score=True, random_state=42, n_jobs=-1 --> .6893
2.   n_estimators = 100 --> .6892, n_estimators = 10 --> .6875, n_estimators = 2000 --> .6892
3.   max_depth = 10 --> .5629, max_depth = 500 --> .6892, max_depth = 50 --> .6894
4.   min_samples_split = 2 --> .8483
5.   min_samples_leaf = 3 --> .6117, min_samples_leaf = 10 --> .5702
6.   max_features doesn't change anything
7.   oob_score = False doesn't change anything
8.   n_jobs = 1 doesn't change anything


In [ ]:
y = police["New Groupings"]
clf_forest = RandomForestClassifier(criterion='gini',
                            max_depth=50,
                            random_state=42)
clf_forest.fit(X, y)
forest_predictions = clf_forest.predict(X)
accuracy_score(y, forest_predictions)

0.855544530742122

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
print("Cross Val Score:", np.mean(cross_val_score(clf_forest,X,y, cv = StratifiedKFold(shuffle=True))))

Cross Val Score: 0.5512197248525055


In [ ]:
#takes a long time to run
for n in range(5):
     strat_k_fold = StratifiedKFold(n_splits=10, shuffle=False, random_state=n)
     scores = cross_val_score(clf_forest, X, y, cv=strat_k_fold)
np.mean(scores)

In [ ]:
from sklearn.model_selection import GridSearchCV
clf_forest = RandomForestClassifier()
clf_forest_grid = GridSearchCV(clf_forest, {'criterion':('gini', 'entropy'), 'max_features': [30, 40]})
clf_forest_grid.fit(X, y)
forest_grid_predictions = clf_forest_grid.predict(X)
accuracy_score(y, forest_grid_predictions)

0.8483765011804154

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
print("Cross Val Score:", np.mean(cross_val_score(clf_forest_grid,X,y, cv = StratifiedKFold(shuffle=True))))

KeyboardInterrupt: ignored

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(50, 10), max_iter=1000)
mlp.fit(X,y)
mlp_predictions = mlp.predict(X)
accuracy_score(y, mlp_predictions)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
print("Cross Val Score:", np.mean(cross_val_score(mlp,X,y, cv = StratifiedKFold(shuffle=True))))